# Process a Stack of Sentinel-1 Burst SLCs with MiaplPy


## Fetch some Sentinel-1 burst SLCs with `burst2safe`

In [ ]:
!burst2safe S1_008704_IW1_20160329T004010_VV_95F5-BURST S1_008703_IW1_20160329T004007_VV_95F5-BURST
!burst2safe S1_008704_IW1_20160317T004009_VV_A84B-BURST S1_008703_IW1_20160317T004006_VV_A84B-BURST
!burst2safe S1_008704_IW1_20160305T004009_VV_659C-BURST S1_008703_IW1_20160305T004006_VV_659C-BURST
!burst2safe S1_008704_IW1_20160210T004009_VV_CB9E-BURST S1_008703_IW1_20160210T004006_VV_CB9E-BURST
!burst2safe S1_008704_IW1_20160129T004009_VV_E8FD-BURST S1_008703_IW1_20160129T004006_VV_E8FD-BURST

## ISCE topsStack's sentinelApp takes SLC zips as inputs and stackSentinel takes SAFE files, so we need both

- Place zips in `SAFE_zip`
- Place SAFE files in `SAFE`

In [1]:
from pathlib import Path

safe_dir = Path.cwd() / "SAFE"
safe_zip_dir = Path.cwd() / "SAFE_zip"
safe_dir.mkdir(exist_ok=True)
safe_zip_dir.mkdir(exist_ok=True)

safe_paths = list(Path.cwd().glob("S1*.SAFE"))
for p in safe_paths:
    zip_pth = safe_zip_dir / f"{p.stem}.zip"
    !zip -r {zip_pth} {p.name}
    p.rename(safe_dir / p.name)

  adding: S1A_IW_SLC__1SSV_20160317T004006_20160317T004009_010402_00F6AB_2AC9.SAFE/ (stored 0%)
  adding: S1A_IW_SLC__1SSV_20160317T004006_20160317T004009_010402_00F6AB_2AC9.SAFE/preview/ (stored 0%)
  adding: S1A_IW_SLC__1SSV_20160317T004006_20160317T004009_010402_00F6AB_2AC9.SAFE/preview/map-overlay.kml (deflated 59%)
  adding: S1A_IW_SLC__1SSV_20160317T004006_20160317T004009_010402_00F6AB_2AC9.SAFE/preview/icons/ (stored 0%)
  adding: S1A_IW_SLC__1SSV_20160317T004006_20160317T004009_010402_00F6AB_2AC9.SAFE/preview/icons/logo.png (deflated 0%)
  adding: S1A_IW_SLC__1SSV_20160317T004006_20160317T004009_010402_00F6AB_2AC9.SAFE/preview/product-preview.html (deflated 77%)
  adding: S1A_IW_SLC__1SSV_20160317T004006_20160317T004009_010402_00F6AB_2AC9.SAFE/support/ (stored 0%)
  adding: S1A_IW_SLC__1SSV_20160317T004006_20160317T004009_010402_00F6AB_2AC9.SAFE/support/s1-object-types.xsd (deflated 90%)
  adding: S1A_IW_SLC__1SSV_20160317T004006_20160317T004009_010402_00F6AB_2AC9.SAFE/support/

## Fetch DEM with the isce dem script

In [1]:
!dem.py -a stitch -b 19 20 -100 -98 -r -s 1 -c

curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N19W100.SRTMGL1.hgt.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (35) OpenSSL SSL_connect: Connection reset by peer in connection to e4ftl01.cr.usgs.gov:443 
2025-07-23 15:49:00,570 - isce.contrib.demUtils.DemStitcher - WARNING - There was a problem in retrieving the file  https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N19W100.SRTMGL1.hgt.zip. Exception 
curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N19W099.SRTMGL1.hgt.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   S

## We need the wgs84 DEM. Delete the UTM DEM

In [2]:
utm_dem = list(Path.cwd().glob("*.dem"))[0]
utm_dem_vrt = list(Path.cwd().glob("*.dem.vrt"))[0]
utm_dem_xml = list(Path.cwd().glob("*.dem.xml"))[0]
utm_dem.unlink()
utm_dem_vrt.unlink()
utm_dem_xml.unlink()      

dem = list(Path.cwd().glob("*.dem.wgs84"))[0]

## Download the AUX_CAL data to an `aux` directory

In [3]:
aux_dir = Path.cwd() / "aux"
aux_dir.mkdir()

!wget https://sar-mpc.eu/download/ca97845e-1314-4817-91d8-f39afbeff74d/ -O {aux_dir}/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.zip

FileExistsError: [Errno 17] File exists: '/home/jovyan/miaplpy-play/2_bursts/aux'

## Run sentinelApp to prep the data for stackSentinel

What exactly is this doing? Converting to `ROI_PAC`?

In [ ]:
cmd = f"sentinelApp.py -s {safe_zip_dir} -o orbits -a {aux_dir} -d {dem} -p vv -n 1"
print(cmd)

Output should look like this:

```bash
Number of SAFE files found: 5
*****************************************
Number of dates : 5
List of dates : 
['20160129', '20160210', '20160305', '20160317', '20160329']
date      south      north      west       east
20160129 19.157758 19.642128 -99.223109 -98.353745
20160210 19.157939 19.641915 -99.224784 -98.357546
20160305 19.158126 19.641989 -99.225818 -98.35861
20160317 19.158336 19.642711 -99.224186 -98.354847
20160329 19.157425 19.641165 -99.225832 -98.358651
*****************************************
All dates
['20160129', '20160210', '20160305', '20160317', '20160329']
No existing processed slc are identified. All the slcs will be processed.
writing  /home/jovyan/miaplpy-play/2_bursts/run_files/run_1
writing  /home/jovyan/miaplpy-play/2_bursts/run_files/run_2
```

## Remove `sentinelApp`'s `run_files` to avoid name collision with `stackSentinel`

In [ ]:
import shutil

run_files_dir = Path.cwd() / "run_files"
shutil.rmtree(run_files_dir)

## Move all the orbit files out of their date directories and into the parent directory

In [5]:
orbits_dir = Path.cwd() / "orbits"
orbit_files = list(orbits_dir.glob("*/*POEORB*"))

for p in orbit_files:
    p.rename(f"{orbits_dir}/{p.name}")
    shutil.rmtree(p.parent)

## Run `stackSentinel` to create the run commands that will prep the data for `Myaplpy`

In [18]:
!stackSentinel.py -h

usage: stackSentinel.py [-h] [-H] -s SLC_DIRNAME -o ORBIT_DIRNAME -a
                        AUX_DIRNAME [-w WORK_DIR] -d DEM [-p POLARIZATION]
                        [-W {slc,correlation,interferogram,offset}]
                        [-n SWATH_NUM] [-b BBOX] [-x EXCLUDE_DATES]
                        [-i INCLUDE_DATES] [--start_date STARTDATE]
                        [--stop_date STOPDATE] [-C {geometry,NESD}]
                        [-m REFERENCE_DATE]
                        [--snr_misreg_threshold SNRTHRESHOLD]
                        [-e ESDCOHERENCETHRESHOLD]
                        [-O NUM_OVERLAP_CONNECTIONS] [-c NUM_CONNECTIONS]
                        [-z AZIMUTHLOOKS] [-r RANGELOOKS] [-f FILTSTRENGTH]
                        [-u {icu,snaphu}] [-rmFilter] [--param_ion PARAM_ION]
                        [--num_connections_ion NUM_CONNECTIONS_ION] [-useGPU]
                        [--num_proc NUMPROCESS]
                        [--num_proc4topo NUMPROCESS4TOPO] [-t TEXT_CMD]
 

In [19]:
cmd = f"stackSentinel.py -s {safe_dir} -o {orbits_dir} -a {aux_dir} -d {dem} -p vv -W slc -n 1"
print(cmd)

stackSentinel.py -s /home/jovyan/miaplpy-play/2_bursts/SAFE -o /home/jovyan/miaplpy-play/2_bursts/orbits -a /home/jovyan/miaplpy-play/2_bursts/aux -d /home/jovyan/miaplpy-play/2_bursts/demLat_N19_N20_Lon_W100_W098.dem.wgs84 -p vv -W slc -n 1


In [20]:
!{cmd}

Number of SAFE files found: 5
Number of SAFE files to be used (cover BBOX): 5
*****************************************
Number of dates : 5
List of dates : 
['20160129', '20160210', '20160305', '20160317', '20160329']
date      south      north
20160129 19.157758 19.642128
20160210 19.157939 19.641915
20160305 19.158126 19.641989
20160317 19.158336 19.642711
20160329 19.157425 19.641165
*****************************************
The overlap region among all dates (based on the preview kml files):
 South   North   East  West 
19.158336 19.641165 -99.223109 -98.358651
*****************************************
All dates (5)
['20160129', '20160210', '20160305', '20160317', '20160329']

The reference date was not chosen. The first date is considered as reference date.

All SLCs will be coregistered to : 20160129
secondary dates :
['20160210', '20160305', '20160317', '20160329']

No existing stack was identified. A new stack will be generated.
*****************************************
Coregis

In [21]:
run_files = sorted(list(Path.cwd().glob("run_files/run*")))
run_files

[PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_01_unpack_topo_reference'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_02_unpack_secondary_slc'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_03_average_baseline'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_04_extract_burst_overlaps'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_05_overlap_geo2rdr'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_06_overlap_resample'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_07_pairs_misreg'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_08_timeseries_misreg'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_09_fullBurst_geo2rdr'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_10_fullBurst_resample'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_files/run_11_extract_stack_valid_region'),
 PosixPath('/home/jovyan/miaplpy-play/2_bursts/run_f

## Make the run files executable and run them

In [22]:
for p in run_files:
    !chmod +x {p}

In [23]:
# run_01_unpack_topo_reference
!bash {run_files[0]}

Completed Parsing the Configuration file
Functions to be executed:
['Function-1', 'Function-2']
Running: Sentinel1_TOPS
['--dirname', '/home/jovyan/miaplpy-play/2_bursts/SAFE/S1A_IW_SLC__1SSV_20160129T004006_20160129T004009_009702_00E294_AD67.SAFE', '--swaths', '1', '--orbitdir', '/home/jovyan/miaplpy-play/2_bursts/orbits', '--outdir', '/home/jovyan/miaplpy-play/2_bursts/reference', '--auxdir', '/home/jovyan/miaplpy-play/2_bursts/aux', '--pol', 'vv']
Input XML files:  ['/home/jovyan/miaplpy-play/2_bursts/SAFE/S1A_IW_SLC__1SSV_20160129T004006_20160129T004009_009702_00E294_AD67.SAFE/annotation/s1a-iw1-slc-vv-20160129t004006-20160129t004012-009702-00e294-001.xml']
Input TIFF files:  ['/home/jovyan/miaplpy-play/2_bursts/SAFE/S1A_IW_SLC__1SSV_20160129T004006_20160129T004009_009702_00E294_AD67.SAFE/measurement/s1a-iw1-slc-vv-20160129t004006-20160129t004012-009702-00e294-001.tiff']
Manifest files:  ['/home/jovyan/miaplpy-play/2_bursts/SAFE/S1A_IW_SLC__1SSV_20160129T004006_20160129T004009_0097

In [24]:
# If no xml annotation files found, you may have included swaths not present in data. Try setting `-n` when running stackSentinel

# run_02_unpack_secondary_slc
!bash {run_files[1]}

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: Sentinel1_TOPS
['--dirname', '/home/jovyan/miaplpy-play/2_bursts/SAFE/S1A_IW_SLC__1SSV_20160210T004006_20160210T004009_009877_00E794_2EFB.SAFE', '--swaths', '1', '--orbitdir', '/home/jovyan/miaplpy-play/2_bursts/orbits', '--outdir', '/home/jovyan/miaplpy-play/2_bursts/secondarys/20160210', '--auxdir', '/home/jovyan/miaplpy-play/2_bursts/aux', '--pol', 'vv']
Input XML files:  ['/home/jovyan/miaplpy-play/2_bursts/SAFE/S1A_IW_SLC__1SSV_20160210T004006_20160210T004009_009877_00E794_2EFB.SAFE/annotation/s1a-iw1-slc-vv-20160210t004006-20160210t004012-009877-00e794-001.xml']
Input TIFF files:  ['/home/jovyan/miaplpy-play/2_bursts/SAFE/S1A_IW_SLC__1SSV_20160210T004006_20160210T004009_009877_00E794_2EFB.SAFE/measurement/s1a-iw1-slc-vv-20160210t004006-20160210t004012-009877-00e794-001.tiff']
Manifest files:  ['/home/jovyan/miaplpy-play/2_bursts/SAFE/S1A_IW_SLC__1SSV_20160210T004006_20160210T004009_009877_0

In [25]:
# run_03_average_baseline
!bash {run_files[2]}

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: computeBaseline
['--reference', '/home/jovyan/miaplpy-play/2_bursts/reference/', '--secondary', '/home/jovyan/miaplpy-play/2_bursts/secondarys/20160210', '--baseline_file', '/home/jovyan/miaplpy-play/2_bursts/baselines/20160129_20160210/20160129_20160210.txt']
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Bprep:  [46.29574569165331, 46.02607876071535]
Bpar:  [-16.895319276622914, -16.724864299835605]
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: computeBaseline
['--reference', '/home/jovyan/miaplpy-play/2_bursts/reference/', '--secondary', '/home/jovyan/miaplpy-play/2_bursts/secondarys/20160305', '--baseline_file', '/home/jovyan/miaplpy-play/2_bursts/baselines/20160129_20160305/20160129_20160305.txt']
minSecondary,maxSecondary 1 2
minReference,maxRe

In [26]:
# run_04_extract_burst_overlaps
!bash {run_files[3]}

Overlap 0 start time - IW-1 2016-01-29 00:40:09.447800 subset
Overlap 0 stop time - IW-1 2016-01-29 00:40:09.749967 subset
Overlap 0 number of lines - IW-1 148 subset
Overlap 0 number of valid lines - IW-1 114 subset
Top:  ['/home/jovyan/miaplpy-play/2_bursts/reference/overlap/IW1/burst_top_01_02.slc']
Bottom:  ['/home/jovyan/miaplpy-play/2_bursts/reference/overlap/IW1/burst_bot_01_02.slc']


In [27]:
# run_05_overlap_geo2rdr
!bash {run_files[4]}

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: geo2rdr
['--secondary', '/home/jovyan/miaplpy-play/2_bursts/secondarys/20160210', '--reference', '/home/jovyan/miaplpy-play/2_bursts/reference', '--geom_referenceDir', '/home/jovyan/miaplpy-play/2_bursts/geom_reference', '--coregSLCdir', '/home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160210', '--overlap']
CPU mode
Estimated burst offset:  0
Overlap pair 0: Burst 0 of reference matched with Burst 0 of secondary
Additional time offset applied in geo2rdr: 0.0 secs
Additional range offset applied in geo2rdr: 0.0 m
Turning off bistatic delay correction term by default.
API open (WR): /home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160210/overlap/IW1/range_top_01_02.off
API open (WR): /home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160210/overlap/IW1/azimuth_top_01_02.off
GDAL open (R): /home/jovyan/miaplpy-play/2_bursts/geom_reference/overlap/IW1/hgt_01_02.rdr.vrt
GDAL open (R): /h

In [28]:
# run_06_overlap_resample
!bash {run_files[5]}

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: resamp_withCarrier
['--secondary', '/home/jovyan/miaplpy-play/2_bursts/secondarys/20160210', '--reference', '/home/jovyan/miaplpy-play/2_bursts/reference', '--coregdir', '/home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160210', '--overlap']
Estimated burst offset:  0
Shifts:  {0: 0, 1: 0}
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Chi squared: 0.000057
Polynomial Order: 5 - by - 3 
13142.2	-699.951	50.3499	1.11562	
-51226.1	2532.51	-144.688	-3.83366	
49917.9	-2277.03	91.4296	2.95043	
5.42188e-09	-5.35067e-09	1.8412e-09	0	
-4.12996e-09	1.50389e-09	0	0	
1.22202e-09	0	0	0	
Misfit radians - Max: 0.00019229298050049692 , Min : -0.0001793507872207556 
True
2025-07-21 18:47:39,076 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_LINES has been set to the default value 1489 which is the number of lines in the slc image.
2025-07-21 18:47:39,076 - isce.stdproc.resamp_sl

In [29]:
# run_07_pairs_misreg
!bash {run_files[6]}

Completed Parsing the Configuration file
Functions to be executed:
['Function-1', 'Function-2', 'Function-3', 'Function-4']
Running: generateIgram
['--reference', '/home/jovyan/miaplpy-play/2_bursts/reference', '--secondary', '/home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160210', '--interferogram', '/home/jovyan/miaplpy-play/2_bursts/coarse_interferograms/20160129_20160210', '--interferogram_prefix', 'int', '--overlap']
topReference.bursts: [<isceobj.Sensor.TOPS.BurstSLC.BurstSLC object at 0x7f98e2924e90>]
minSecondary,maxSecondary 1 1
minReference,maxReference 1 1
minBurst, maxBurst:  1 1
matching burst numbers:  1 1
multiply
/home/jovyan/.local/envs/miaplpy-env/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
read
No range offsets provided
rdict:  {'rangeOff1': '/home/jovyan/miaplpy-play/2_bursts/reference/ov

In [30]:
# run_08_timeseries_misreg
!bash {run_files[7]}


Rank of design matrix: 4
Design matrix is full rank.
RMSE : 7.785021135790039e-05 pixels

Estimated offsets with respect to the stack reference date

20160129 : 0.0
20160210 : -0.00021083145965323033
20160305 : -0.0009726043523849658
20160317 : -0.0009338402426237072
20160329 : 0.0006640897166982694


Rank of design matrix: 4
Design matrix is full rank.
RMSE : 0.01687044100706511 pixels

Estimated offsets with respect to the stack reference date

20160129 : 0.0
20160210 : 0.21354320044350494
20160305 : 0.24266272839614816
20160317 : 0.12618461658557537
20160329 : 0.19413018153721276



In [31]:
# run_09_fullBurst_geo2rdr
!bash {run_files[8]}

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: geo2rdr
['--secondary', '/home/jovyan/miaplpy-play/2_bursts/secondarys/20160210', '--reference', '/home/jovyan/miaplpy-play/2_bursts/reference', '--geom_referenceDir', '/home/jovyan/miaplpy-play/2_bursts/geom_reference', '--coregSLCdir', '/home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160210', '--azimuth_misreg', '/home/jovyan/miaplpy-play/2_bursts/misreg/azimuth/dates/20160210.txt', '--range_misreg', '/home/jovyan/miaplpy-play/2_bursts/misreg/range/dates/20160210.txt']
CPU mode
Estimated burst offset:  0
Overlap pair 0: Burst 0 of reference matched with Burst 0 of secondary
Burst 0 of reference matched with Burst 0 of secondary
Additional time offset applied in geo2rdr: -4.33375935128393e-07 secs
Additional range offset applied in geo2rdr: 0.21354320044350494 m
Turning off bistatic delay correction term by default.
API open (WR): /home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/2016021

In [32]:
# run_10_fullBurst_resample
!bash {run_files[9]}

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: resamp_withCarrier
['--secondary', '/home/jovyan/miaplpy-play/2_bursts/secondarys/20160210', '--reference', '/home/jovyan/miaplpy-play/2_bursts/reference', '--coregdir', '/home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160210', '--azimuth_misreg', '/home/jovyan/miaplpy-play/2_bursts/misreg/azimuth/dates/20160210.txt', '--range_misreg', '/home/jovyan/miaplpy-play/2_bursts/misreg/range/dates/20160210.txt']
Estimated burst offset:  0
Shifts:  {0: 0, 1: 0}
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Chi squared: 0.000057
Polynomial Order: 5 - by - 3 
13142.2	-699.951	50.3499	1.11562	
-51226.1	2532.51	-144.688	-3.83366	
49917.9	-2277.03	91.4296	2.95043	
5.42188e-09	-5.35067e-09	1.8412e-09	0	
-4.12996e-09	1.50389e-09	0	0	
1.22202e-09	0	0	0	
Misfit radians - Max: 0.00019229298050049692 , Min : -0.0001793507872207556 
True
2025-07-21 19:03:20,214 - isce.stdproc.resamp_slc - WA

In [33]:
# run_11_extract_stack_valid_region
!bash {run_files[10]}

creating  /home/jovyan/miaplpy-play/2_bursts/stack
checking the number of bursts in coreg_secondarys against the one in reference
all secondary images have the same number of bursts as the reference
******************
swath:  1
/home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160210
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
/home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160305
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
/home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160317
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
/home/jovyan/miaplpy-play/2_bursts/coreg_secondarys/20160329
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
writing  /home/jovyan/miaplpy-play/2_bursts/stack/IW1.xml


In [34]:
# run_12_merge_reference_secondary_slc
!bash {run_files[11]}

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/home/jovyan/miaplpy-play/2_bursts/stack', '--inp_reference', '/home/jovyan/miaplpy-play/2_bursts/reference', '--dirname', '/home/jovyan/miaplpy-play/2_bursts/reference', '--name_pattern', 'burst*slc', '--outfile', '/home/jovyan/miaplpy-play/2_bursts/merged/SLC/20160129/20160129.slc', '--method', 'top', '--valid_only', '--range_looks', '9', '--azimuth_looks', '3']
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 2
writing merged file to disk via gdal.Translate ...
/home/jovyan/.local/envs/miaplpy-env/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
00..11....30..40..50..61...71..80..100 
Skipping multi-looking ....
Completed Parsing the Configuration file
Functions

In [35]:
# run_13_grid_baseline
!bash {run_files[12]}

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: baselineGrid
['--reference', '/home/jovyan/miaplpy-play/2_bursts/reference/', '--secondary', '/home/jovyan/miaplpy-play/2_bursts/secondarys/20160210', '--baseline_file', '/home/jovyan/miaplpy-play/2_bursts/merged/baselines/20160210/20160210']
Baseline file /home/jovyan/miaplpy-play/2_bursts/merged/baselines/20160210/20160210 dims: 6L x 7P
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynom

## Create a `miaplpy` config

In [2]:
miaplpy_config = """

################
miaplpy.load.processor      = isce  #[isce,snap,gamma,roipac], auto for isceTops
miaplpy.load.updateMode     = auto  #[yes / no], auto for yes, skip re-loading if HDF5 files are complete
miaplpy.load.compression    = gzip  #[gzip / lzf / no], auto for no.
miaplpy.load.autoPath       = yes    # [yes, no] auto for no
        
miaplpy.load.slcFile        = ../merged/SLC/*/*.slc.full  #[path2slc_file]
##---------for ISCE only:
miaplpy.load.metaFile       = ../reference/IW*.xml
miaplpy.load.baselineDir    = ../baselines
##---------geometry datasets:
miaplpy.load.demFile          = ../merged/geom_reference/hgt.rdr.full
miaplpy.load.lookupYFile      = ../merged/geom_reference/lat.rdr.full
miaplpy.load.lookupXFile      = ../merged/geom_reference/lon.rdr.full
miaplpy.load.incAngleFile     = ../merged/geom_reference/los.rdr.full
miaplpy.load.azAngleFile      = ../merged/geom_reference/los.rdr.full
miaplpy.load.shadowMaskFile   = ../merged/geom_reference/shadowMask.rdr.full
miaplpy.load.waterMaskFile    = None
##---------interferogram datasets:
miaplpy.load.unwFile        = ./inverted/interferograms_single_reference/*/*fine*.unw
miaplpy.load.corFile        = ./inverted/interferograms_single_reference/*/*fine*.cor
miaplpy.load.connCompFile   = ./inverted/interferograms_single_reference/*/*.unw.conncomp

# MiaplPy options 
miaplpy.multiprocessing.numProcessor   = 40
miaplpy.interferograms.type = single_reference

## Mintpy options
mintpy.compute.cluster     = local  # if dask is not available, set this option to no 
mintpy.compute.numWorker   = 8 

mintpy.troposphericDelay.method = no
"""

In [4]:
from pathlib import Path

config_path = Path.cwd() / "miaplpy_config.txt"
config_path.write_text(miaplpy_config)

1610

## Run `miaplpyApp.py`

In [6]:
#  File "/home/jovyan/tools/MiaplPy/src/miaplpy/unwrap_ifgram.py", line 24, in <module>
#     from isceobj.Util.ImageUtil import ImageLib as IML
# ModuleNotFoundError: No module named 'isceobj'

# Must import isce prior to import isceobj


!miaplpyApp.py {config_path} --dir ./miaplpy

Using default MintPy Path: /home/jovyan/tools/MiaplPy/src
--------------------------------------------------

_________________________________________________      

  /##      /## /##                     / ## /#######
 | ###    /###|__/|                    | ##| ##__  ##
 | ####  /#### /##|/ ##### /##         | ##| ##  \ ## /##   /##
 | ## ##/## ##| ##| ##__  ### /####### | ##| #######/| ##  | ##
 | ##  ###| ##| ##| ##__  ###| ##__  ##| ##| ##      | ##  | ##
 | ##\  # | ##| ##| ##   \###| ##   \##| ##| ##      | ##  | ##
 | ## \/  | ##| ##|  ##### ##| #######/| ##| ##      |  #######
 |__/     |__/|__/| \____/|_/| ##____/ |__/|__/       \____  ##
                             | ##                     /##  | ## 
                             | ##                    |  ######/         
                             |__/                     \______/
 Miami Non-Linear Phase Linking software in Python   
          MiaplPy 0.2.1, 2023-08-22
_________________________________________________

